# Plan: Tweet Sentiment Extraction (Medal-Oriented)

Objectives:
- Establish reliable CV and a fast baseline ASAP
- Train a strong span-extraction transformer (QA-style) with start/end heads
- Apply robust post-processing (especially for neutral → full tweet, punctuation trimming, whitespace fixes)
- Ensemble diverse seeds/folds; calibrate with OOF
- Iterate via error analysis

Validation:
- Use StratifiedKFold by sentiment (5 folds) with deterministic seeds
- Fit transforms inside folds; cache features/logits
- Score OOF via Jaccard on reconstructed text

Baselines and Iteration:
1) Smoke-check environment (GPU) and data integrity
2) Heuristic baseline:
   - neutral → tweet_text
   - positive/negative → simple char-span heuristic centered on sentiment words (regex-based), fallback to tweet_text
   - Expect ~0.63–0.67 Jaccard (sanity gate)
3) Transformer QA model:
   - Input: "question" = sentiment; "context" = tweet
   - Tokenizer: RoBERTa-base (byte-level BPE) or DeBERTa-v3-base
   - Max length ~96–128; pad/truncate only context
   - Loss: cross-entropy on start/end; label smoothing 0.05
   - Optimizer: AdamW, lr warmup, cosine decay; epochs ~3–5 with early stop
   - Folds: 5; save start/end logits per fold for OOF/test; average logits
   - Post-process: ensure start<=end, neutral full span, detokenize with offsets, trim spaces/punctuation only if improves Jaccard on OOF
   - Target Jaccard: ≥0.715 OOF before ensembling; ≥0.72 with careful PP/ensembles
4) Ensembling:
   - Blend seeds (2–3) per backbone
   - Consider two backbones (RoBERTa-base + DeBERTa-v3-base) if time
   - Weighted average of logits; weights chosen by OOF

Risk Controls / Checks:
- Always print fold progress and elapsed time
- Cache tokenized dataset and offsets
- Verify submission.csv format and encoding

Milestones (with expert reviews):
A) Plan review (this cell)
B) Data loading + EDA snapshot
C) Baseline heuristic + CV
D) Transformer v1 (single backbone, 5-fold) + PP v1
E) Error analysis + PP v2
F) Ensembling seeds/backbones
G) Final checks and submission

In [1]:
import os, sys, time, json, textwrap, math, random, statistics as stats, subprocess
import pandas as pd
import numpy as np

print('=== GPU CHECK ===', flush=True)
try:
    out = subprocess.run(['bash','-lc','nvidia-smi || true'], capture_output=True, text=True, check=False)
    print(out.stdout)
except Exception as e:
    print('nvidia-smi failed:', e)

print('Python', sys.version)
print('CWD:', os.getcwd(), flush=True)

train_path = 'train.csv'
test_path = 'test.csv'
ss_path = 'sample_submission.csv'
for p in [train_path, test_path, ss_path]:
    print(p, os.path.exists(p), os.path.getsize(p) if os.path.exists(p) else None)

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
ss = pd.read_csv(ss_path)

print('\n=== Head(train) ===')
print(train.head(3).to_string(index=False))
print('\n=== Head(test) ===')
print(test.head(3).to_string(index=False))
print('\n=== Head(sample_submission) ===')
print(ss.head(3).to_string(index=False))

print('\nShapes:', train.shape, test.shape)
print('Columns(train):', list(train.columns))
print('Columns(test):', list(test.columns))

# Basic checks
print('\nNulls in train:')
print(train.isnull().sum())
print('\nSentiment distribution (train):')
print(train['sentiment'].value_counts())

train['tweet_len'] = train['text'].astype(str).str.len()
train['sel_len'] = train['selected_text'].astype(str).str.len()
print('\nTweet length stats:', train['tweet_len'].describe(percentiles=[0.5,0.9,0.95,0.99]).to_dict())
print('Selected length stats:', train['sel_len'].describe(percentiles=[0.5,0.9,0.95,0.99]).to_dict())

def jaccard(a, b):
    a = str(a)
    b = str(b)
    a_set = set(a.split())
    b_set = set(b.split())
    if not a_set and not b_set:
        return 1.0
    if not a_set or not b_set:
        return 0.0
    inter = len(a_set & b_set)
    union = len(a_set | b_set)
    return inter / union if union else 0.0

# Quick sanity: compute jaccard of gold vs itself
jac_self = train.apply(lambda r: jaccard(r['selected_text'], r['selected_text']), axis=1).mean()
print('\nSanity Jaccard(selected vs selected):', jac_self)

print('\nSample rows:')
for i in range(3):
    r = train.sample(1, random_state=42+i).iloc[0]
    print({'id': r['textID'], 'sentiment': r['sentiment'], 'text': r['text'][:120], 'selected': r['selected_text']})

print('\nDone EDA snapshot.', flush=True)

=== GPU CHECK ===


Tue Sep 30 15:43:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

train.csv True 3151814
test.csv True 244504
sample_submission.csv True 33009

=== Head(train) ===
    textID                                                                         text                                      selected_text sentiment
8d4ad58b45                           eating breakfast  getting ready to go to school ;( eating breakfast  getting ready to go to school ;(  negative
fdfe12a800 Going to fold laundry and then hit the sack. I have boring saturday evenings                    I have boring saturday evenings  negative
5efd224f4e            happy mothers day to all   im off to spend the day with my family                                              happy  positive

=== Head(test) ===
    textID                                                                                                                          text sentiment
80a1e6bc32                                                                                  I just saw a shooting star... I made my wish  p


Sanity Jaccard(selected vs selected): 1.0

Sample rows:


{'id': '62da1d2932', 'sentiment': 'positive', 'text': ' Thanks for the advice! Went to the doctor`s and slept a lot yesterday  Must be the meds.', 'selected': 'Thanks'}
{'id': '8027499c43', 'sentiment': 'positive', 'text': ' I absolutely LOVE you.  Thanks', 'selected': 'I absolutely LOVE you.  Thanks'}
{'id': 'eeb70d82b3', 'sentiment': 'negative', 'text': 'i`m so tired  of being sick ALL the time!!!!', 'selected': 'i`m so tired'}

Done EDA snapshot.


In [2]:
# Heuristic baseline: build quick OOF estimate and a valid submission
import re

pos_keywords = [
    'love','lovely','awesome','amazing','great','good','best','glad','happy','yay','thanks','thank you','excited',
    ':)',':-)',':D','<3','lol','lmao','rofl'
]
neg_keywords = [
    "n't",'not','no ',' never','sad','bad','worse','worst','hate','angry','upset','sucks','tired','sick','terrible','awful',
    ':(',' :-(',' :/',' :|'
]

def find_keyword_span(text: str, kws):
    if not isinstance(text, str) or not text:
        return None
    low = text.lower()
    for kw in kws:
        kw_low = kw.lower()
        idx = low.find(kw_low)
        if idx != -1:
            return text[idx: idx + len(kw)]
    return None

def heuristic_selected(text: str, sentiment: str):
    if not isinstance(text, str) or not text:
        return ''
    s = (sentiment or '').strip().lower()
    if s == 'neutral':
        return text
    if s == 'positive':
        span = find_keyword_span(text, pos_keywords)
        return span if span else text
    if s == 'negative':
        span = find_keyword_span(text, neg_keywords)
        return span if span else text
    return text

# OOF heuristic score on train
train_pred = train.apply(lambda r: heuristic_selected(r['text'], r['sentiment']), axis=1)
heuristic_oof = train.apply(lambda r: jaccard(r['selected_text'], heuristic_selected(r['text'], r['sentiment'])), axis=1).mean()
print(f'Heuristic OOF Jaccard (sanity): {heuristic_oof:.4f}')

# Build submission
sub = test.copy()
sub['selected_text'] = sub.apply(lambda r: heuristic_selected(r['text'], r['sentiment']), axis=1)
submission_path = 'submission.csv'
sub[['textID','selected_text']].to_csv(submission_path, index=False)
print('Wrote', submission_path, 'Head:')
print(sub[['textID','selected_text']].head().to_string(index=False))

Heuristic OOF Jaccard (sanity): 0.5902
Wrote submission.csv Head:
    textID                                                                                       selected_text
80a1e6bc32                                                        I just saw a shooting star... I made my wish
863097735d                                                                                               upset
264cd5277f             tired and didn`t really have an exciting Saturday.  oh well, hope it`s better tomorrow.
baee1e6ffc                                                              i`ve been eating cheetos all morning..
67d06a8dee  haiiii sankQ i`m fineee ima js get a checkup cos my rib hurts LOL idk but i shall be fine ~ thanks


In [3]:
# Install cu121 torch stack and core NLP deps; sanity-check GPU
import os, sys, subprocess, shutil, time

def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Uninstall any pre-existing torch stacks to avoid duplicates
for pkg in ('torch','torchvision','torchaudio'):
    try:
        subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)
    except Exception as e:
        print('Uninstall warn:', pkg, e)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d, flush=True)
        shutil.rmtree(d, ignore_errors=True)

# Install exact cu121 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# Constraints to pin torch versions for later installs
from pathlib import Path
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# Install NLP deps honoring constraints
pip('install', '-c', 'constraints.txt',
    'transformers==4.44.2', 'accelerate==0.34.2',
    'datasets==2.21.0', 'evaluate==0.4.2',
    'sentencepiece', 'scikit-learn', 'tqdm',
    '--upgrade-strategy', 'only-if-needed')

import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None), flush=True)
print('CUDA available:', torch.cuda.is_available(), flush=True)
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0), flush=True)

> pip install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 415.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 517.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 351.7 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 KB 552.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 KB 509.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 KB 568.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 559.2 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1


CUDA available: True


GPU: NVIDIA A10-24Q


In [4]:
# QA setup: tokenizer, CV folds, and alignment sanity checks
import re, numpy as np, random, time
from sklearn.model_selection import StratifiedKFold
from transformers import AutoTokenizer

SEED = 42
def seed_everything(seed=SEED):
    random.seed(seed); np.random.seed(seed)
seed_everything()

# Drop nulls for label creation
train_clean = train.dropna(subset=['text','selected_text']).reset_index(drop=True)

# Create stratified folds by sentiment
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
folds = np.full(len(train_clean), -1, dtype=int)
for f, (_, val_idx) in enumerate(skf.split(train_clean, train_clean['sentiment'])):
    folds[val_idx] = f
train_clean['fold'] = folds
print('Folds assigned:', np.bincount(folds))

model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

IGNORE_INDEX = -100
def tokenize_and_align(tokenizer, text, sentiment, selected_text=None, max_len=128):
    enc = tokenizer(
        str(sentiment), str(text),
        max_length=max_len, padding='max_length',
        truncation='only_second', add_special_tokens=True,
        return_offsets_mapping=True, return_attention_mask=True
    )
    offsets = enc['offset_mapping']
    seq_ids = enc.sequence_ids()
    ctx_idx = [i for i, sid in enumerate(seq_ids) if sid == 1]
    start_pos = end_pos = IGNORE_INDEX
    if isinstance(selected_text, str) and isinstance(text, str) and ctx_idx:
        # find all occurrences; choose the one with max token-char overlap
        matches = [m.start() for m in re.finditer(re.escape(selected_text), text)]
        if matches:
            best = None
            for st_char in matches:
                ed_char = st_char + len(selected_text)
                overlap = 0
                for i in ctx_idx:
                    a, b = offsets[i]
                    overlap += max(0, min(ed_char, b) - max(st_char, a))
                if (best is None) or (overlap > best[0]):
                    best = (overlap, st_char, ed_char)
            _, st_char, ed_char = best
            chosen = [i for i in ctx_idx
                      if offsets[i][1] > offsets[i][0] and
                         max(offsets[i][0], st_char) < min(offsets[i][1], ed_char)]
            if chosen:
                start_pos, end_pos = chosen[0], chosen[-1]
            else:
                dists = [(abs(offsets[i][0]-st_char)+abs(offsets[i][1]-ed_char), i)
                         for i in ctx_idx if offsets[i][1] > offsets[i][0]]
                if dists:
                    start_pos = end_pos = min(dists)[1]
        # else: keep IGNORE_INDEX (rare)
    enc['start_positions'] = start_pos
    enc['end_positions'] = end_pos
    return enc, ctx_idx

# Sanity check a few samples per sentiment and fold
def reconstruct_from_positions(text, offsets, i, j):
    if i < 0 or j < 0: return ''
    s_char, e_char = offsets[i][0], offsets[j][1]
    return text[s_char:e_char]

samples_checked = 0
for s in ['neutral','positive','negative']:
    df_s = train_clean[train_clean.sentiment==s].head(2)
    for _, r in df_s.iterrows():
        enc, ctx = tokenize_and_align(tokenizer, r['text'], r['sentiment'], r['selected_text'], max_len=128)
        pred_span = reconstruct_from_positions(r['text'], enc['offset_mapping'], enc['start_positions'], enc['end_positions'])
        print({'sent': r['sentiment'], 'text_snip': r['text'][:60], 'gold': r['selected_text'], 'recon': pred_span})
        samples_checked += 1
print('Alignment samples printed:', samples_checked)

print('Setup OK. Next: implement model/train loop with 5-fold QA and OOF logging.', flush=True)

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Folds assigned: [4947 4946 4946 4946 4946]


{'sent': 'neutral', 'text_snip': '  not Pimm`s in a can?', 'gold': 'not Pimm`s in a can?', 'recon': 'not Pimm`s in a can?'}
{'sent': 'neutral', 'text_snip': ' i would, but i don`t know how to do it from the phone...', 'gold': 'i would, but i don`t know how to do it from the phone...', 'recon': 'i would, but i don`t know how to do it from the phone...'}
{'sent': 'positive', 'text_snip': 'happy mothers day to all   im off to spend the day with my f', 'gold': 'happy', 'recon': 'happy'}
{'sent': 'positive', 'text_snip': ' one of my favorite quotes ever', 'gold': 'favorite', 'recon': 'favorite'}
{'sent': 'negative', 'text_snip': 'eating breakfast  getting ready to go to school ;(', 'gold': 'eating breakfast  getting ready to go to school ;(', 'recon': 'eating breakfast  getting ready to go to school ;('}
{'sent': 'negative', 'text_snip': 'Going to fold laundry and then hit the sack. I have boring s', 'gold': 'I have boring saturday evenings', 'recon': 'I have boring saturday evenings'}
Alig

/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# 5-fold QA training with RoBERTa-base, AMP, OOF logging/caching
import os, gc, math, json, time, numpy as np, pandas as pd, torch
from dataclasses import dataclass
from typing import Optional, Dict, Any, List, Tuple
from transformers import (
    AutoModelForQuestionAnswering,
    Trainer, TrainingArguments,
    default_data_collator,
    get_cosine_schedule_with_warmup,
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 3
LR = 2e-5
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
GRAD_CLIP = 1.0

def build_encodings(df: pd.DataFrame, include_labels: bool = True):
    enc_list = []
    for i, r in df.iterrows():
        sel = r['selected_text'] if include_labels else None
        enc, ctx_idx = tokenize_and_align(tokenizer, r['text'], r['sentiment'], sel, max_len=MAX_LEN)
        # Persist needed fields for eval/decoding
        enc['text'] = r['text']
        enc['sentiment'] = r['sentiment']
        enc_list.append(enc)
    # Stack into arrays
    keys = enc_list[0].keys()
    out: Dict[str, Any] = {}
    for k in keys:
        vals = [e[k] for e in enc_list]
        if k in ('text','sentiment'):
            out[k] = vals
        else:
            out[k] = np.array(vals, dtype=object if k=='offset_mapping' else None)
    return out

class QADataset(torch.utils.data.Dataset):
    def __init__(self, enc: Dict[str, Any], with_labels: bool):
        self.enc = enc
        self.with_labels = with_labels
    def __len__(self):
        return len(self.enc['input_ids'])
    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.enc['input_ids'][idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.enc['attention_mask'][idx], dtype=torch.long),
        }
        if self.with_labels:
            item['start_positions'] = torch.tensor(self.enc['start_positions'][idx], dtype=torch.long)
            item['end_positions'] = torch.tensor(self.enc['end_positions'][idx], dtype=torch.long)
        return item

def get_sequence_ids_for_pair(sentiment: str, text: str):
    tmp = tokenizer(str(sentiment), str(text),
                    max_length=MAX_LEN, padding='max_length', truncation='only_second',
                    add_special_tokens=True, return_offsets_mapping=True)
    return tmp.sequence_ids(), tmp['offset_mapping']

def decode_one(start_logits, end_logits, offsets, sequence_ids, text, sentiment, length_penalty_per_char=0.0, low_conf_threshold=None):
    if str(sentiment).strip().lower() == 'neutral':
        return text
    ctx = [i for i, sid in enumerate(sequence_ids) if sid == 1]
    if not ctx:
        return text
    sl = np.asarray(start_logits, dtype=np.float32)
    el = np.asarray(end_logits, dtype=np.float32)
    mask = np.zeros_like(sl, dtype=bool)
    mask[np.array(ctx)] = True
    sl[~mask] = -np.inf; el[~mask] = -np.inf
    best_score = -1e9; bi = bj = ctx[0]
    for i in ctx:
        for j in ctx:
            if j < i: continue
            span_len_chars = offsets[j][1] - offsets[i][0]
            score = sl[i] + el[j] - length_penalty_per_char * span_len_chars
            if score > best_score:
                best_score, bi, bj = score, i, j
    if (low_conf_threshold is not None) and (best_score < low_conf_threshold):
        return text
    s_char, e_char = offsets[bi][0], offsets[bj][1]
    pred = text[s_char:e_char].strip()
    return pred if pred else text

def jaccard_batch(trues: List[str], preds: List[str]):
    return float(np.mean([jaccard(t, p) for t, p in zip(trues, preds)]))

oof_rows = []
start_logits_folds = []
end_logits_folds = []

for fold in range(5):
    t0 = time.time()
    print(f'\n===== Fold {fold} =====', flush=True)
    trn_df = train_clean[train_clean.fold != fold].reset_index(drop=True)
    val_df = train_clean[train_clean.fold == fold].reset_index(drop=True)
    print('Train/Val sizes:', len(trn_df), len(val_df))

    trn_enc = build_encodings(trn_df, include_labels=True)
    val_enc = build_encodings(val_df, include_labels=True)

    train_ds = QADataset(trn_enc, with_labels=True)
    val_ds = QADataset(val_enc, with_labels=True)

    model = AutoModelForQuestionAnswering.from_pretrained('roberta-base')

    total_steps = math.ceil(len(train_ds) / (BATCH_SIZE*2)) * EPOCHS

    # Compute metrics closure will do decoding on val set
    def compute_metrics(eval_pred):
        start_logits, end_logits = eval_pred.predictions
        preds = []
        trues = list(val_df['selected_text'].astype(str).values)
        for i in range(len(val_df)):
            text = val_df.iloc[i]['text']
            sentiment = val_df.iloc[i]['sentiment']
            seq_ids, offs = get_sequence_ids_for_pair(sentiment, text)
            pred_text = decode_one(start_logits[i], end_logits[i], offs, seq_ids, text, sentiment,
                                   length_penalty_per_char=0.003, low_conf_threshold=None)
            preds.append(pred_text)
        score = jaccard_batch(trues, preds)
        return {'jaccard': score}

    args = TrainingArguments(
        output_dir=f'./outputs_fold{fold}',
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='jaccard',
        greater_is_better=True,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE*2,
        gradient_accumulation_steps=2,
        num_train_epochs=EPOCHS,
        fp16=True,
        learning_rate=LR,
        weight_decay=WEIGHT_DECAY,
        warmup_ratio=WARMUP_RATIO,
        lr_scheduler_type='cosine',
        max_grad_norm=GRAD_CLIP,
        dataloader_num_workers=2,
        logging_steps=50,
        save_total_limit=1,
        seed=SEED,
        report_to=[]
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics
    )

    train_out = trainer.train()
    print('Best model metrics:', train_out.metrics, flush=True)
    # Save best model for this fold
    trainer.save_model(f'fold{fold}_best')
    # Record best checkpoint path
    if getattr(trainer.state, 'best_model_checkpoint', None):
        with open(f'fold{fold}_best/path.txt', 'w') as f:
            f.write(trainer.state.best_model_checkpoint)

    # Inference on val to get logits for caching and OOF decode
    val_preds = trainer.predict(val_ds)
    val_start_logits, val_end_logits = val_preds.predictions
    start_logits_folds.append(val_start_logits)
    end_logits_folds.append(val_end_logits)

    # Decode OOF
    val_trues = list(val_df['selected_text'].astype(str).values)
    val_preds_text = []
    for i in range(len(val_df)):
        text = val_df.iloc[i]['text']
        sentiment = val_df.iloc[i]['sentiment']
        seq_ids, offs = get_sequence_ids_for_pair(sentiment, text)
        pred_text = decode_one(val_start_logits[i], val_end_logits[i], offs, seq_ids, text, sentiment,
                               length_penalty_per_char=0.003, low_conf_threshold=None)
        val_preds_text.append(pred_text)
        oof_rows.append({
            'textID': val_df.iloc[i]['textID'],
            'fold': fold,
            'sentiment': sentiment,
            'text': text,
            'selected_text': val_trues[i],
            'pred': pred_text
        })
    fold_j = jaccard_batch(val_trues, val_preds_text)
    print(f'Fold {fold} OOF Jaccard: {fold_j:.5f}; elapsed {time.time()-t0:.1f}s', flush=True)

    # Cleanup
    del trainer, model, train_ds, val_ds, trn_enc, val_enc
    gc.collect(); torch.cuda.empty_cache()

# Aggregate OOF
oof_df = pd.DataFrame(oof_rows)
oof_score = jaccard_batch(oof_df['selected_text'].tolist(), oof_df['pred'].tolist())
print(f'OOF Jaccard (all folds): {oof_score:.5f}', flush=True)
oof_df.to_csv('oof_roberta_base.csv', index=False)
np.save('oof_start_logits_roberta_base.npy', np.concatenate(start_logits_folds, axis=0))
np.save('oof_end_logits_roberta_base.npy', np.concatenate(end_logits_folds, axis=0))
print('Saved OOF artifacts.')

# Predict on test with the best single model per fold and average logits across folds
test_df = test.copy().reset_index(drop=True)
test_enc_list = []
for i, r in test_df.iterrows():
    enc = tokenizer(
        str(r['sentiment']), str(r['text']),
        max_length=MAX_LEN, padding='max_length', truncation='only_second',
        add_special_tokens=True, return_offsets_mapping=True, return_attention_mask=True
    )
    test_enc_list.append(enc)
test_input_ids = torch.tensor([e['input_ids'] for e in test_enc_list], dtype=torch.long)
test_attention_mask = torch.tensor([e['attention_mask'] for e in test_enc_list], dtype=torch.long)

all_fold_test_start = []
all_fold_test_end = []
for fold in range(5):
    print(f'Test inference with fold {fold} checkpoint...', flush=True)
    model = AutoModelForQuestionAnswering.from_pretrained(f'fold{fold}_best').to(device)
    model.eval()
    with torch.no_grad():
        bs = BATCH_SIZE
        starts, ends = [], []
        for i in range(0, len(test_df), bs):
            input_ids = test_input_ids[i:i+bs].to(device)
            attn = test_attention_mask[i:i+bs].to(device)
            out = model(input_ids=input_ids, attention_mask=attn)
            starts.append(out.start_logits.detach().cpu().numpy())
            ends.append(out.end_logits.detach().cpu().numpy())
        starts = np.vstack(starts); ends = np.vstack(ends)
    # cache per-fold test logits
    np.save(f'test_start_fold{fold}.npy', starts)
    np.save(f'test_end_fold{fold}.npy', ends)
    all_fold_test_start.append(starts)
    all_fold_test_end.append(ends)
    del model; gc.collect(); torch.cuda.empty_cache()

avg_test_start = np.mean(all_fold_test_start, axis=0)
avg_test_end = np.mean(all_fold_test_end, axis=0)

# Decode test
test_preds = []
for i in range(len(test_df)):
    text = test_df.iloc[i]['text']
    sentiment = test_df.iloc[i]['sentiment']
    seq_ids, offs = get_sequence_ids_for_pair(sentiment, text)
    pred_text = decode_one(avg_test_start[i], avg_test_end[i], offs, seq_ids, text, sentiment,
                           length_penalty_per_char=0.003, low_conf_threshold=None)
    test_preds.append(pred_text)
sub = pd.DataFrame({'textID': test_df['textID'], 'selected_text': test_preds})
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv (model). Head:\n', sub.head().to_string(index=False), flush=True)


===== Fold 0 =====


Train/Val sizes: 19784 4947


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/app/.pip-target/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/app/.pip-target/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Test inference with fold 1 checkpoint...


Test inference with fold 2 checkpoint...


Test inference with fold 3 checkpoint...


Test inference with fold 4 checkpoint...


Wrote submission.csv (model). Head:
     textID                                                                                       selected_text
80a1e6bc32                                                                                      I made my wish
863097735d                                                                                              sucks!
264cd5277f             tired and didn`t really have an exciting Saturday.  oh well, hope it`s better tomorrow.
baee1e6ffc                                                              i`ve been eating cheetos all morning..
67d06a8dee  haiiii sankQ i`m fineee ima js get a checkup cos my rib hurts LOL idk but i shall be fine ~ thanks


In [6]:
# DeBERTa-v3-base 5-fold QA training (expect +0.005–0.01 OOF vs RoBERTa), same pipeline
import os, gc, math, json, time, numpy as np, pandas as pd, torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments, default_data_collator

MODEL_NAME = 'microsoft/deberta-v3-base'
print('Loading tokenizer/model:', MODEL_NAME, flush=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 3
LR = 2e-5
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
GRAD_CLIP = 1.0

def build_encodings_df(df: pd.DataFrame, include_labels: bool = True):
    enc_list = []
    for _, r in df.iterrows():
        sel = r['selected_text'] if include_labels else None
        enc, _ = tokenize_and_align(tokenizer, r['text'], r['sentiment'], sel, max_len=MAX_LEN)
        enc['text'] = r['text']
        enc['sentiment'] = r['sentiment']
        enc_list.append(enc)
    keys = enc_list[0].keys()
    out = {}
    for k in keys:
        vals = [e[k] for e in enc_list]
        if k in ('text','sentiment'): out[k] = vals
        else: out[k] = np.array(vals, dtype=object if k=='offset_mapping' else None)
    return out

class QADataset2(torch.utils.data.Dataset):
    def __init__(self, enc, with_labels): self.enc, self.with_labels = enc, with_labels
    def __len__(self): return len(self.enc['input_ids'])
    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.enc['input_ids'][idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.enc['attention_mask'][idx], dtype=torch.long),
        }
        if self.with_labels:
            item['start_positions'] = torch.tensor(self.enc['start_positions'][idx], dtype=torch.long)
            item['end_positions'] = torch.tensor(self.enc['end_positions'][idx], dtype=torch.long)
        return item

def get_seq_ids_offsets(sentiment: str, text: str):
    tmp = tokenizer(str(sentiment), str(text), max_length=MAX_LEN, padding='max_length', truncation='only_second',
                    add_special_tokens=True, return_offsets_mapping=True)
    return tmp.sequence_ids(), tmp['offset_mapping']

def decode_span(start_logits, end_logits, offsets, sequence_ids, text, sentiment, length_penalty_per_char=0.003):
    if str(sentiment).strip().lower() == 'neutral': return text
    ctx = [i for i, sid in enumerate(sequence_ids) if sid == 1]
    if not ctx: return text
    sl = np.asarray(start_logits, dtype=np.float32); el = np.asarray(end_logits, dtype=np.float32)
    mask = np.zeros_like(sl, dtype=bool); mask[np.array(ctx)] = True
    sl[~mask] = -np.inf; el[~mask] = -np.inf
    best, bi, bj = -1e9, ctx[0], ctx[0]
    for i in ctx:
        for j in ctx:
            if j < i: continue
            span_len = offsets[j][1] - offsets[i][0]
            sc = sl[i] + el[j] - length_penalty_per_char * span_len
            if sc > best: best, bi, bj = sc, i, j
    s_char, e_char = offsets[bi][0], offsets[bj][1]
    pred = text[s_char:e_char].strip()
    return pred if pred else text

def jaccard_batch_fast(trues, preds):
    return float(np.mean([jaccard(t, p) for t, p in zip(trues, preds)]))

oof_rows2, start_logits_folds2, end_logits_folds2 = [], [], []
for fold in range(5):
    t0 = time.time(); print(f'\n===== DeBERTa Fold {fold} =====', flush=True)
    trn_df = train_clean[train_clean.fold != fold].reset_index(drop=True)
    val_df = train_clean[train_clean.fold == fold].reset_index(drop=True)
    print('Train/Val sizes:', len(trn_df), len(val_df))

    trn_enc = build_encodings_df(trn_df, include_labels=True)
    val_enc = build_encodings_df(val_df, include_labels=True)
    train_ds = QADataset2(trn_enc, with_labels=True)
    val_ds = QADataset2(val_enc, with_labels=True)

    model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

    def compute_metrics(eval_pred):
        start_logits, end_logits = eval_pred.predictions
        preds, trues = [], list(val_df['selected_text'].astype(str).values)
        for i in range(len(val_df)):
            text, sentiment = val_df.iloc[i]['text'], val_df.iloc[i]['sentiment']
            seq_ids, offs = get_seq_ids_offsets(sentiment, text)
            preds.append(decode_span(start_logits[i], end_logits[i], offs, seq_ids, text, sentiment))
        return {'jaccard': jaccard_batch_fast(trues, preds)}

    args = TrainingArguments(
        output_dir=f'./outputs_{MODEL_NAME.replace("/","_")}_fold{fold}',
        evaluation_strategy='epoch', save_strategy='epoch',
        load_best_model_at_end=True, metric_for_best_model='jaccard', greater_is_better=True,
        per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2,
        gradient_accumulation_steps=2, num_train_epochs=EPOCHS, fp16=True,
        learning_rate=LR, weight_decay=WEIGHT_DECAY, warmup_ratio=WARMUP_RATIO, lr_scheduler_type='cosine',
        max_grad_norm=GRAD_CLIP, dataloader_num_workers=2, logging_steps=50, save_total_limit=1, seed=SEED, report_to=[]
    )

    trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds,
                      tokenizer=tokenizer, data_collator=default_data_collator, compute_metrics=compute_metrics)
    train_out = trainer.train()
    print('Best model metrics:', train_out.metrics, flush=True)
    save_dir = f'deberta_fold{fold}_best'
    trainer.save_model(save_dir)
    if getattr(trainer.state, 'best_model_checkpoint', None):
        with open(os.path.join(save_dir, 'path.txt'), 'w') as f: f.write(trainer.state.best_model_checkpoint)

    val_preds = trainer.predict(val_ds)
    vsl, vel = val_preds.predictions
    start_logits_folds2.append(vsl); end_logits_folds2.append(vel)

    trues = list(val_df['selected_text'].astype(str).values)
    preds = []
    for i in range(len(val_df)):
        text, sentiment = val_df.iloc[i]['text'], val_df.iloc[i]['sentiment']
        seq_ids, offs = get_seq_ids_offsets(sentiment, text)
        preds.append(decode_span(vsl[i], vel[i], offs, seq_ids, text, sentiment))
        oof_rows2.append({
            'textID': val_df.iloc[i]['textID'], 'fold': fold, 'sentiment': sentiment, 'text': text,
            'selected_text': trues[i], 'pred': preds[-1]
        })
    fj = jaccard_batch_fast(trues, preds)
    print(f'DeBERTa Fold {fold} OOF Jaccard: {fj:.5f}; elapsed {time.time()-t0:.1f}s', flush=True)

    del trainer, model, train_ds, val_ds, trn_enc, val_enc
    gc.collect(); torch.cuda.empty_cache()

oof_df2 = pd.DataFrame(oof_rows2)
oof_score2 = jaccard_batch_fast(oof_df2['selected_text'].tolist(), oof_df2['pred'].tolist())
print(f'DeBERTa OOF Jaccard (all folds): {oof_score2:.5f}', flush=True)
oof_df2.to_csv('oof_deberta_v3_base.csv', index=False)
np.save('oof_start_logits_deberta_v3_base.npy', np.concatenate(start_logits_folds2, axis=0))
np.save('oof_end_logits_deberta_v3_base.npy', np.concatenate(end_logits_folds2, axis=0))
print('Saved DeBERTa OOF artifacts.')

# Test inference with best fold checkpoints and average logits
test_df = test.copy().reset_index(drop=True)
test_enc = [tokenizer(str(r['sentiment']), str(r['text']), max_length=MAX_LEN, padding='max_length',
                     truncation='only_second', add_special_tokens=True, return_offsets_mapping=True, return_attention_mask=True)
            for _, r in test_df.iterrows()]
test_input_ids = torch.tensor([e['input_ids'] for e in test_enc], dtype=torch.long)
test_attention_mask = torch.tensor([e['attention_mask'] for e in test_enc], dtype=torch.long)
fold_starts, fold_ends = [], []
for fold in range(5):
    print(f'DeBERTa test inference fold {fold}...', flush=True)
    model = AutoModelForQuestionAnswering.from_pretrained(f'deberta_fold{fold}_best').to(device); model.eval()
    with torch.no_grad():
        bs = BATCH_SIZE; starts, ends = [], []
        for i in range(0, len(test_df), bs):
            out = model(input_ids=test_input_ids[i:i+bs].to(device), attention_mask=test_attention_mask[i:i+bs].to(device))
            starts.append(out.start_logits.detach().cpu().numpy()); ends.append(out.end_logits.detach().cpu().numpy())
        starts, ends = np.vstack(starts), np.vstack(ends)
    np.save(f'deberta_test_start_fold{fold}.npy', starts); np.save(f'deberta_test_end_fold{fold}.npy', ends)
    fold_starts.append(starts); fold_ends.append(ends)
    del model; gc.collect(); torch.cuda.empty_cache()

avg_st = np.mean(fold_starts, axis=0); avg_en = np.mean(fold_ends, axis=0)
test_preds = []
for i in range(len(test_df)):
    text, sentiment = test_df.iloc[i]['text'], test_df.iloc[i]['sentiment']
    seq_ids, offs = get_seq_ids_offsets(sentiment, text)
    test_preds.append(decode_span(avg_st[i], avg_en[i], offs, seq_ids, text, sentiment))
sub2 = pd.DataFrame({'textID': test_df['textID'], 'selected_text': test_preds})
sub2.to_csv('submission_deberta.csv', index=False)
print('Wrote submission_deberta.csv Head:\n', sub2.head().to_string(index=False))
print('DeBERTa run complete.')

Loading tokenizer/model: microsoft/deberta-v3-base


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/.pip-target/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(



===== DeBERTa Fold 0 =====


Train/Val sizes: 19784 4947


Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

DeBERTa test inference fold 1...


DeBERTa test inference fold 2...


DeBERTa test inference fold 3...


DeBERTa test inference fold 4...


Wrote submission_deberta.csv Head:
     textID                                                                                       selected_text
80a1e6bc32                                                                                                wish
863097735d                                                                                   gosh today sucks!
264cd5277f             tired and didn`t really have an exciting Saturday.  oh well, hope it`s better tomorrow.
baee1e6ffc                                                              i`ve been eating cheetos all morning..
67d06a8dee  haiiii sankQ i`m fineee ima js get a checkup cos my rib hurts LOL idk but i shall be fine ~ thanks
DeBERTa run complete.


In [7]:
# Post-processing sweep on RoBERTa OOF logits; apply tuned PP to test
import numpy as np, pandas as pd, time
from transformers import AutoTokenizer

print('PP tuning on RoBERTa OOF...', flush=True)
oof_df = pd.read_csv('oof_roberta_base.csv')
oof_sl = np.load('oof_start_logits_roberta_base.npy')
oof_el = np.load('oof_end_logits_roberta_base.npy')
assert len(oof_df) == oof_sl.shape[0] == oof_el.shape[0], 'Mismatch OOF shapes'

rb_tokenizer = AutoTokenizer.from_pretrained('roberta-base', use_fast=True)

def seq_ids_offs_rb(sentiment, text, max_len=128):
    tmp = rb_tokenizer(str(sentiment), str(text), max_length=max_len, padding='max_length',
                       truncation='only_second', add_special_tokens=True, return_offsets_mapping=True)
    return tmp.sequence_ids(), tmp['offset_mapping']

def decode_with_score(sl, el, offs, seq_ids, text, sentiment, lp=0.0):
    if str(sentiment).strip().lower() == 'neutral':
        return text, 0.0
    ctx = [i for i, sid in enumerate(seq_ids) if sid == 1]
    if not ctx: return text, -1e9
    sl = np.asarray(sl, dtype=np.float32); el = np.asarray(el, dtype=np.float32)
    mask = np.zeros_like(sl, dtype=bool); mask[np.array(ctx)] = True
    sl[~mask] = -np.inf; el[~mask] = -np.inf
    best, bi, bj = -1e9, ctx[0], ctx[0]
    for i in ctx:
        for j in ctx:
            if j < i: continue
            span_len = offs[j][1] - offs[i][0]
            sc = sl[i] + el[j] - lp * float(span_len)
            if sc > best: best, bi, bj = sc, i, j
    s_char, e_char = offs[bi][0], offs[bj][1]
    pred = text[s_char:e_char].strip()
    if not pred: pred = text
    return pred, float(best)

def jaccard_mean(y_true, y_pred):
    def jac(a,b):
        sa, sb = set(str(a).split()), set(str(b).split())
        if not sa and not sb: return 1.0
        if not sa or not sb: return 0.0
        inter = len(sa & sb); union = len(sa | sb)
        return inter/union if union else 0.0
    return float(np.mean([jac(t,p) for t,p in zip(y_true, y_pred)]))

def eval_params(lp, fallback_pct=None):
    preds = []; scores = []
    t0 = time.time()
    for i in range(len(oof_df)):
        r = oof_df.iloc[i]
        seq_ids, offs = seq_ids_offs_rb(r['sentiment'], r['text'])
        pred, sc = decode_with_score(oof_sl[i], oof_el[i], offs, seq_ids, r['text'], r['sentiment'], lp=lp)
        preds.append(pred); scores.append(sc)
    if fallback_pct is not None:
        thr = np.percentile(scores, fallback_pct*100.0)
        for i in range(len(oof_df)):
            if scores[i] < thr:
                # fallback: return full text for any sentiment (neutral already full)
                preds[i] = oof_df.iloc[i]['text']
    score = jaccard_mean(oof_df['selected_text'].tolist(), preds)
    return score, preds, scores

grid_lp = [0.0, 0.002, 0.003, 0.004, 0.006, 0.008]
grid_fb = [None, 0.02, 0.05]
best = (-1, None, None)
for lp in grid_lp:
    for fb in grid_fb:
        s, _, _ = eval_params(lp, fb)
        print(f'lp={lp:.4f} fb={fb} -> OOF {s:.5f}')
        if s > best[0]: best = (s, lp, fb)
print('Best PP:', best)

# Apply best PP to test using saved per-fold test logits
test_df = pd.read_csv('test.csv')
fold_starts = []; fold_ends = []
for f in range(5):
    fold_starts.append(np.load(f'test_start_fold{f}.npy'))
    fold_ends.append(np.load(f'test_end_fold{f}.npy'))
avg_st = np.mean(fold_starts, axis=0); avg_en = np.mean(fold_ends, axis=0)

best_oof, best_lp, best_fb = best
print(f'Using tuned params: lp={best_lp} fb={best_fb}', flush=True)

test_preds = []
if best_fb is not None:
    # derive threshold from OOF distribution and reuse on test: use same percentile of OOF scores as threshold value
    _, _, oof_scores = eval_params(best_lp, None)
    thr = np.percentile(oof_scores, best_fb*100.0)
else:
    thr = None
for i in range(len(test_df)):
    text = test_df.iloc[i]['text']; sent = test_df.iloc[i]['sentiment']
    seq_ids, offs = seq_ids_offs_rb(sent, text)
    pred, sc = decode_with_score(avg_st[i], avg_en[i], offs, seq_ids, text, sent, lp=best_lp)
    if (thr is not None) and (sc < thr):
        pred = text
    test_preds.append(pred)

sub_pp = pd.DataFrame({'textID': test_df['textID'], 'selected_text': test_preds})
sub_pp.to_csv('submission_pp_roberta.csv', index=False)
print('Wrote submission_pp_roberta.csv. Head:\n', sub_pp.head().to_string(index=False))
print('PP tuning complete.')

PP tuning on RoBERTa OOF...


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


lp=0.0000 fb=None -> OOF 0.70694


lp=0.0000 fb=0.02 -> OOF 0.70694


lp=0.0000 fb=0.05 -> OOF 0.70694


...

lp=0.0060 fb=0.05 -> OOF 0.70854


lp=0.0080 fb=None -> OOF 0.70887


lp=0.0080 fb=0.02 -> OOF 0.70887


lp=0.0080 fb=0.05 -> OOF 0.70887
Best PP: (0.7088670288691741, 0.008, None)
Using tuned params: lp=0.008 fb=None


Wrote submission_pp_roberta.csv. Head:
     textID                                                                                       selected_text
80a1e6bc32                                                                                      I made my wish
863097735d                                                                                              sucks!
264cd5277f             tired and didn`t really have an exciting Saturday.  oh well, hope it`s better tomorrow.
baee1e6ffc                                                              i`ve been eating cheetos all morning..
67d06a8dee  haiiii sankQ i`m fineee ima js get a checkup cos my rib hurts LOL idk but i shall be fine ~ thanks
PP tuning complete.


In [8]:
# PP tuning and blending utilities (staged). Run after DeBERTa training finishes.
import os, json, time, math, numpy as np, pandas as pd
from typing import Dict, Tuple, Optional
from transformers import AutoTokenizer

print('Staging PP tuning and blending code. Do not run until all OOF/test logits are saved.', flush=True)

MAX_LEN = 128

def jaccard_str(a: str, b: str) -> float:
    sa, sb = set(str(a).split()), set(str(b).split())
    if not sa and not sb: return 1.0
    if not sa or not sb: return 0.0
    inter = len(sa & sb); union = len(sa | sb)
    return inter/union if union else 0.0

def jaccard_mean(y_true, y_pred):
    return float(np.mean([jaccard_str(t, p) for t, p in zip(y_true, y_pred)]))

def load_oof(model_key: str) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray, AutoTokenizer]:
    # model_key in {'roberta_base','deberta_v3_base'}
    if model_key == 'roberta_base':
        oof_df = pd.read_csv('oof_roberta_base.csv')
        sl = np.load('oof_start_logits_roberta_base.npy'); el = np.load('oof_end_logits_roberta_base.npy')
        tok = AutoTokenizer.from_pretrained('roberta-base', use_fast=True)
    elif model_key == 'deberta_v3_base':
        oof_df = pd.read_csv('oof_deberta_v3_base.csv')
        sl = np.load('oof_start_logits_deberta_v3_base.npy'); el = np.load('oof_end_logits_deberta_v3_base.npy')
        tok = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base', use_fast=True)
    else:
        raise ValueError('Unknown model_key')
    assert len(oof_df) == sl.shape[0] == el.shape[0], f'Shape mismatch for {model_key}'
    return oof_df, sl, el, tok

def get_seq_ids_offs(tok: AutoTokenizer, sentiment: str, text: str):
    tmp = tok(str(sentiment), str(text), max_length=MAX_LEN, padding='max_length', truncation='only_second',
              add_special_tokens=True, return_offsets_mapping=True)
    return tmp.sequence_ids(), tmp['offset_mapping']

def decode_with_score(sl_row: np.ndarray, el_row: np.ndarray, offs, seq_ids, text: str, sentiment: str, lp_char: float) -> Tuple[str, float]:
    # Guards and neutral rule
    if str(sentiment).strip().lower() == 'neutral':
        return text, 0.0
    ctx = [i for i, sid in enumerate(seq_ids) if sid == 1]
    if not ctx: return text, -1e9
    sl = np.asarray(sl_row, dtype=np.float32).copy()
    el = np.asarray(el_row, dtype=np.float32).copy()
    mask = np.zeros_like(sl, dtype=bool); mask[np.array(ctx)] = True
    sl[~mask] = -np.inf; el[~mask] = -np.inf
    best, bi, bj = -1e9, ctx[0], ctx[0]
    # Ignore zero-length offsets
    valid = [i for i in ctx if offs[i][1] > offs[i][0]]
    if not valid: return text, -1e9
    for i in valid:
        for j in valid:
            if j < i: continue
            span_len = offs[j][1] - offs[i][0]
            sc = float(sl[i]) + float(el[j]) - lp_char * float(span_len)
            if sc > best:
                best, bi, bj = sc, i, j
    s_char, e_char = offs[bi][0], offs[bj][1]
    pred = text[s_char:e_char].strip()
    if not pred: pred = text
    return pred, float(best)

def eval_oof_with_params(oof_df: pd.DataFrame, sl: np.ndarray, el: np.ndarray, tok: AutoTokenizer,
                          lp_by_sent: Dict[str, float], fb_pct_by_sent: Dict[str, Optional[float]]):
    preds, scores = [], []
    for i in range(len(oof_df)):
        r = oof_df.iloc[i]
        sent = str(r['sentiment']).strip().lower()
        seq_ids, offs = get_seq_ids_offs(tok, r['sentiment'], r['text'])
        lp = lp_by_sent.get(sent, 0.0)
        pred, sc = decode_with_score(sl[i], el[i], offs, seq_ids, r['text'], r['sentiment'], lp_char=lp)
        preds.append(pred); scores.append(sc)
    # Apply low-confidence fallback per sentiment for pos/neg only
    for s_key in ('positive','negative'):
        pct = fb_pct_by_sent.get(s_key, None)
        if pct is None: continue
        idxs = [i for i in range(len(oof_df)) if str(oof_df.iloc[i]['sentiment']).strip().lower()==s_key]
        if not idxs: continue
        thr = np.percentile([scores[i] for i in idxs], pct*100.0)
        for i in idxs:
            if scores[i] < thr:
                preds[i] = oof_df.iloc[i]['text']
    score = jaccard_mean(oof_df['selected_text'].tolist(), preds)
    return score, preds, scores

def tune_pp_for_model(model_key: str, lp_grid=(0.0, 0.002, 0.003, 0.004, 0.006, 0.008), fb_grid=(None, 0.02, 0.05)):
    oof_df, sl, el, tok = load_oof(model_key)
    best = (-1.0, None)
    for lp_pos in lp_grid:
        for lp_neg in lp_grid:
            lp_by_sent = {'positive': lp_pos, 'negative': lp_neg}
            for fb_pos in fb_grid:
                for fb_neg in fb_grid:
                    fb_by_sent = {'positive': fb_pos, 'negative': fb_neg}
                    s, _, _ = eval_oof_with_params(oof_df, sl, el, tok, lp_by_sent, fb_by_sent)
                    print(f'[{model_key}] lp(pos)={lp_pos:.4f} lp(neg)={lp_neg:.4f} fb(pos)={fb_pos} fb(neg)={fb_neg} -> OOF {s:.5f}')
                    if s > best[0]:
                        best = (s, (lp_by_sent, fb_by_sent))
    print(f'Best {model_key} PP:', best)
    return best, (oof_df, sl, el, tok)

def load_test_logits(model_key: str) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray, AutoTokenizer]:
    test_df = pd.read_csv('test.csv')
    if model_key == 'roberta_base':
        st = np.mean([np.load(f'test_start_fold{f}.npy') for f in range(5)], axis=0)
        en = np.mean([np.load(f'test_end_fold{f}.npy') for f in range(5)], axis=0)
        tok = AutoTokenizer.from_pretrained('roberta-base', use_fast=True)
    elif model_key == 'deberta_v3_base':
        st = np.mean([np.load(f'deberta_test_start_fold{f}.npy') for f in range(5)], axis=0)
        en = np.mean([np.load(f'deberta_test_end_fold{f}.npy') for f in range(5)], axis=0)
        tok = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base', use_fast=True)
    else:
        raise ValueError('Unknown model_key')
    assert st.shape[0] == len(test_df) == en.shape[0], 'Test shape mismatch'
    return test_df, st, en, tok

def apply_pp_to_test(model_key: str, lp_by_sent: Dict[str, float], fb_pct_by_sent: Dict[str, Optional[float]],
                     ref_oof_scores_by_sent: Dict[str, list], out_path: str):
    test_df, st, en, tok = load_test_logits(model_key)
    # Derive numeric thresholds from OOF score distributions per sentiment (reuse same percentiles)
    thr_by_sent: Dict[str, Optional[float]] = {}
    for s_key, pct in fb_pct_by_sent.items():
        if pct is None: thr_by_sent[s_key] = None
        else:
            vals = ref_oof_scores_by_sent.get(s_key, [])
            thr_by_sent[s_key] = float(np.percentile(vals, pct*100.0)) if len(vals) else None
    preds = []
    for i in range(len(test_df)):
        text, sent = test_df.iloc[i]['text'], test_df.iloc[i]['sentiment']
        s_key = str(sent).strip().lower()
        if s_key == 'neutral':
            preds.append(text); continue
        seq_ids, offs = get_seq_ids_offs(tok, sent, text)
        pred, sc = decode_with_score(st[i], en[i], offs, seq_ids, text, sent, lp_char=lp_by_sent.get(s_key, 0.0))
        thr = thr_by_sent.get(s_key, None)
        if (thr is not None) and (sc < thr): pred = text
        preds.append(pred)
    sub = pd.DataFrame({'textID': test_df['textID'], 'selected_text': preds})
    sub.to_csv(out_path, index=False)
    print('Wrote', out_path, 'Head:\n', sub.head().to_string(index=False))

def try_blend_oof(w: float,
                  oof_rb: Tuple[pd.DataFrame, np.ndarray, np.ndarray, AutoTokenizer],
                  oof_deb: Tuple[pd.DataFrame, np.ndarray, np.ndarray, AutoTokenizer],
                  lp_grid, fb_grid):
    df_r, sl_r, el_r, tok_r = oof_rb
    df_d, sl_d, el_d, tok_d = oof_deb
    # Sanity: ordering must match; we created folds deterministically so row order of OOFs should match original train_clean order
    assert len(df_r)==len(df_d), 'OOF length mismatch between models'
    # Use tokenizer of one backbone for decoding; we must use the same tokenization setup we will use at test time.
    # Decode with each model's tokenizer produces slightly different offsets; to avoid mismatch, decode using each model's tokenizer separately is not feasible when blending logits.
    # Instead, blend logits at token index level requires identical tokenization, which we do not have across backbones.
    # Therefore, we blend at score level by aligning via recomputing logits per model during test; for OOF, we approximate by averaging per-position logits after re-tokenization is NOT possible.
    # Pragmatic approach: align by using the model whose tokenizer we will decode with for both (not strictly aligned across models).
    # Empirically, blending raw logits from different tokenizers is not valid; instead, we will blend predictions at span-score level:
    # For each candidate span (i,j) over the chosen tokenizer, compute score as w*s_d + (1-w)*s_r, where s_* = start[i] + end[j].
    # To support that, we need start/end logits from both models mapped to the same tokenization. Without re-running models, we cannot remap.
    # Hence: switch to submission-time blend by averaging decoded spans is suboptimal; Experts advised blending logits before decode within same tokenization.
    # Conclusion: Do NOT attempt cross-tokenizer logit blending offline here. We'll perform blending per-backbone separately and pick DeBERTa as primary,
    # and provide optional simple text-level blend (choose shorter/longer span) only if needed.
    raise NotImplementedError('Cross-backbone logit blending requires identical tokenization; stage left as future work with re-forward if needed.')

if __name__ == '__main__':
    print(' Guidance: ',
          '\n1) Run tune_pp_for_model("roberta_base") to get best params and OOF scores per sentiment. Save JSON.',
          '\n2) After DeBERTa finishes and its OOF logits exist, run tune_pp_for_model("deberta_v3_base").',
          '\n3) Apply to test with apply_pp_to_test using thresholds derived from respective OOF.',
          '\n4) For cross-backbone blending at logit level, re-forward both models on test using a shared tokenization is required; otherwise, prefer DeBERTa-only for primary submission.', flush=True)
    # Example manual run after training completes:
    # best_rb, pack_rb = tune_pp_for_model('roberta_base')
    # (score_rb, (lp_rb, fb_rb)) = best_rb
    # # Derive OOF score distributions per sentiment for thresholds
    # oof_df_rb, sl_rb, el_rb, tok_rb = pack_rb
    # _, preds_rb, scores_rb = eval_oof_with_params(oof_df_rb, sl_rb, el_rb, tok_rb, lp_rb, fb_rb)
    # scores_by_sent_rb = {'positive': [scores_rb[i] for i in range(len(oof_df_rb)) if str(oof_df_rb.iloc[i]['sentiment']).strip().lower()=='positive'],
    #                      'negative': [scores_rb[i] for i in range(len(oof_df_rb)) if str(oof_df_rb.iloc[i]['sentiment']).strip().lower()=='negative']}
    # with open('pp_params_roberta.json','w') as f: json.dump({'lp': lp_rb, 'fb': fb_rb, 'oof_score': score_rb}, f)
    # apply_pp_to_test('roberta_base', lp_rb, fb_rb, scores_by_sent_rb, 'submission_pp_roberta.csv')

    # # After DeBERTa OOF exists:
    # best_deb, pack_deb = tune_pp_for_model('deberta_v3_base')
    # (score_deb, (lp_deb, fb_deb)) = best_deb
    # oof_df_deb, sl_deb, el_deb, tok_deb = pack_deb
    # _, preds_deb, scores_deb = eval_oof_with_params(oof_df_deb, sl_deb, el_deb, tok_deb, lp_deb, fb_deb)
    # scores_by_sent_deb = {'positive': [scores_deb[i] for i in range(len(oof_df_deb)) if str(oof_df_deb.iloc[i]['sentiment']).strip().lower()=='positive'],
    #                       'negative': [scores_deb[i] for i in range(len(oof_df_deb)) if str(oof_df_deb.iloc[i]['sentiment']).strip().lower()=='negative']}
    # with open('pp_params_deberta.json','w') as f: json.dump({'lp': lp_deb, 'fb': fb_deb, 'oof_score': score_deb}, f)
    # apply_pp_to_test('deberta_v3_base', lp_deb, fb_deb, scores_by_sent_deb, 'submission_pp_deberta.csv')

    print('PP utilities ready.')

Staging PP tuning and blending code. Do not run until all OOF/test logits are saved.


 Guidance:  
1) Run tune_pp_for_model("roberta_base") to get best params and OOF scores per sentiment. Save JSON. 
2) After DeBERTa finishes and its OOF logits exist, run tune_pp_for_model("deberta_v3_base"). 
3) Apply to test with apply_pp_to_test using thresholds derived from respective OOF. 
4) For cross-backbone blending at logit level, re-forward both models on test using a shared tokenization is required; otherwise, prefer DeBERTa-only for primary submission.


PP utilities ready.


In [9]:
# Run PP tuning for DeBERTa, derive per-sentiment thresholds, and write submission
import json, numpy as np, pandas as pd
print('Tuning PP for DeBERTa-v3-base...', flush=True)
best_deb, pack_deb = tune_pp_for_model('deberta_v3_base')
(score_deb, (lp_deb, fb_deb)) = best_deb
print('Best DeBERTa OOF:', score_deb, 'params:', lp_deb, fb_deb)
oof_df_deb, sl_deb, el_deb, tok_deb = pack_deb
# Get OOF preds and scores to calibrate thresholds
oof_score_eval, preds_deb, scores_deb = eval_oof_with_params(oof_df_deb, sl_deb, el_deb, tok_deb, lp_deb, fb_deb)
scores_by_sent_deb = {
    'positive': [scores_deb[i] for i in range(len(oof_df_deb)) if str(oof_df_deb.iloc[i]['sentiment']).strip().lower()=='positive'],
    'negative': [scores_deb[i] for i in range(len(oof_df_deb)) if str(oof_df_deb.iloc[i]['sentiment']).strip().lower()=='negative']
}
with open('pp_params_deberta.json','w') as f:
    json.dump({'lp': lp_deb, 'fb': fb_deb, 'oof_score': float(score_deb)}, f)
print('Saved pp_params_deberta.json')
apply_pp_to_test('deberta_v3_base', lp_deb, fb_deb, scores_by_sent_deb, 'submission_pp_deberta.csv')
print('DeBERTa PP tuning + submission complete.')

# Optionally, save DeBERTa OOF decoded predictions for error analysis
pd.DataFrame({
    'textID': oof_df_deb['textID'],
    'sentiment': oof_df_deb['sentiment'],
    'text': oof_df_deb['text'],
    'selected_text': oof_df_deb['selected_text'],
    'pred': preds_deb
}).to_csv('oof_preds_deberta_pp.csv', index=False)
print('Saved oof_preds_deberta_pp.csv')

Tuning PP for DeBERTa-v3-base...


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/.pip-target/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[deberta_v3_base] lp(pos)=0.0000 lp(neg)=0.0000 fb(pos)=None fb(neg)=None -> OOF 0.71116


[deberta_v3_base] lp(pos)=0.0000 lp(neg)=0.0000 fb(pos)=None fb(neg)=0.02 -> OOF 0.71089


[deberta_v3_base] lp(pos)=0.0000 lp(neg)=0.0000 fb(pos)=None fb(neg)=0.05 -> OOF 0.71039


...

[deberta_v3_base] lp(pos)=0.0080 lp(neg)=0.0080 fb(pos)=0.05 fb(neg)=0.02 -> OOF 0.71081


[deberta_v3_base] lp(pos)=0.0080 lp(neg)=0.0080 fb(pos)=0.05 fb(neg)=0.05 -> OOF 0.71043
Best deberta_v3_base PP: (0.712386302687562, ({'positive': 0.008, 'negative': 0.002}, {'positive': None, 'negative': None}))
Best DeBERTa OOF: 0.712386302687562 params: {'positive': 0.008, 'negative': 0.002} {'positive': None, 'negative': None}


Saved pp_params_deberta.json


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/.pip-target/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Wrote submission_pp_deberta.csv Head:
     textID                                                                                       selected_text
80a1e6bc32                                                                                                wish
863097735d                                                                                   gosh today sucks!
264cd5277f             tired and didn`t really have an exciting Saturday.  oh well, hope it`s better tomorrow.
baee1e6ffc                                                              i`ve been eating cheetos all morning..
67d06a8dee  haiiii sankQ i`m fineee ima js get a checkup cos my rib hurts LOL idk but i shall be fine ~ thanks
DeBERTa PP tuning + submission complete.
Saved oof_preds_deberta_pp.csv


In [10]:
# Set final submission from DeBERTa PP
import pandas as pd, os
src = 'submission_pp_deberta.csv'
dst = 'submission.csv'
assert os.path.exists(src), f'Missing {src}. Run DeBERTa PP tuning first.'
pd.read_csv(src).to_csv(dst, index=False)
print(f'Wrote {dst} from {src}. Head:')
print(pd.read_csv(dst).head().to_string(index=False))

Wrote submission.csv from submission_pp_deberta.csv. Head:
    textID                                                                                       selected_text
80a1e6bc32                                                                                                wish
863097735d                                                                                   gosh today sucks!
264cd5277f             tired and didn`t really have an exciting Saturday.  oh well, hope it`s better tomorrow.
baee1e6ffc                                                              i`ve been eating cheetos all morning..
67d06a8dee  haiiii sankQ i`m fineee ima js get a checkup cos my rib hurts LOL idk but i shall be fine ~ thanks


In [11]:
# OOF-calibrated chooser ensemble: pick model with higher standardized span score per row
import numpy as np, pandas as pd, json
from transformers import AutoTokenizer

print('Building chooser ensemble (OOF-calibrated z-scores)...', flush=True)

# Load OOF for both models and decode with tuned PP to get per-row scores
rb_oof = pd.read_csv('oof_roberta_base.csv')
rb_sl = np.load('oof_start_logits_roberta_base.npy')
rb_el = np.load('oof_end_logits_roberta_base.npy')
rb_tok = AutoTokenizer.from_pretrained('roberta-base', use_fast=True)

deb_oof = pd.read_csv('oof_deberta_v3_base.csv')
deb_sl = np.load('oof_start_logits_deberta_v3_base.npy')
deb_el = np.load('oof_end_logits_deberta_v3_base.npy')
deb_tok = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base', use_fast=True)

def seq_ids_offs(tok, sentiment, text, max_len=128):
    tmp = tok(str(sentiment), str(text), max_length=max_len, padding='max_length', truncation='only_second',
               add_special_tokens=True, return_offsets_mapping=True)
    return tmp.sequence_ids(), tmp['offset_mapping']

def decode_score(sl, el, offs, seq_ids, text, sentiment, lp):
    if str(sentiment).strip().lower()=='neutral':
        return text, 0.0
    ctx = [i for i,sid in enumerate(seq_ids) if sid==1]
    if not ctx: return text, -1e9
    sl = np.asarray(sl, dtype=np.float32); el = np.asarray(el, dtype=np.float32)
    mask = np.zeros_like(sl, dtype=bool); mask[np.array(ctx)] = True
    sl[~mask] = -np.inf; el[~mask] = -np.inf
    best, bi, bj = -1e9, ctx[0], ctx[0]
    # ignore zero-length offsets
    valid = [i for i in ctx if offs[i][1] > offs[i][0]]
    if not valid: return text, -1e9
    for i in valid:
        for j in valid:
            if j < i: continue
            span_len = offs[j][1] - offs[i][0]
            sc = float(sl[i]) + float(el[j]) - float(lp) * float(span_len)
            if sc > best: best, bi, bj = sc, i, j
    s_char, e_char = offs[bi][0], offs[bj][1]
    pred = text[s_char:e_char].strip()
    if not pred: pred = text
    return pred, float(best)

# Tuned PP params discovered earlier
rb_lp = 0.008  # from Cell 7 best
deb_lp_by_sent = {'positive': 0.008, 'negative': 0.002}  # from Cell 9

def jaccard(a,b):
    sa, sb = set(str(a).split()), set(str(b).split())
    if not sa and not sb: return 1.0
    if not sa or not sb: return 0.0
    inter = len(sa & sb); union = len(sa | sb)
    return inter/union if union else 0.0

# Compute OOF per-row scores for both models
assert len(rb_oof)==len(deb_oof), 'OOF length mismatch; cannot align chooser reliably'
N = len(rb_oof)
rb_scores = np.zeros(N, dtype=np.float32)
deb_scores = np.zeros(N, dtype=np.float32)
rb_preds = ['']*N
deb_preds = ['']*N
trues = rb_oof['selected_text'].astype(str).tolist()
for i in range(N):
    # RoBERTa
    text = rb_oof.iloc[i]['text']; sent = rb_oof.iloc[i]['sentiment']
    sids, offs = seq_ids_offs(rb_tok, sent, text)
    p, sc = decode_score(rb_sl[i], rb_el[i], offs, sids, text, sent, rb_lp)
    rb_scores[i] = sc; rb_preds[i] = p
    # DeBERTa
    text2 = deb_oof.iloc[i]['text']; sent2 = deb_oof.iloc[i]['sentiment']
    sids2, offs2 = seq_ids_offs(deb_tok, sent2, text2)
    lp = deb_lp_by_sent.get(str(sent2).strip().lower(), 0.0)
    p2, sc2 = decode_score(deb_sl[i], deb_el[i], offs2, sids2, text2, sent2, lp)
    deb_scores[i] = sc2; deb_preds[i] = p2

# Standardize scores per sentiment and model using OOF
sents = rb_oof['sentiment'].astype(str).str.lower().tolist()
stats_rb = {}; stats_deb = {}
for s in ['positive','negative']:
    idx = [i for i,x in enumerate(sents) if x==s]
    if idx:
        mu_rb = float(np.mean(rb_scores[idx])); sd_rb = float(np.std(rb_scores[idx]) + 1e-6)
        mu_deb = float(np.mean(deb_scores[idx])); sd_deb = float(np.std(deb_scores[idx]) + 1e-6)
        stats_rb[s] = (mu_rb, sd_rb); stats_deb[s] = (mu_deb, sd_deb)

def zscore(sc, mu_sd):
    if mu_sd is None: return sc
    mu, sd = mu_sd; return (sc - mu) / sd

# Evaluate chooser on OOF: pick model with higher z-score
chooser_preds = []
for i in range(N):
    s = sents[i]
    if s=='neutral':
        chooser_preds.append(rb_oof.iloc[i]['text']);
        continue
    z_rb = zscore(rb_scores[i], stats_rb.get(s, None))
    z_deb = zscore(deb_scores[i], stats_deb.get(s, None))
    chooser_preds.append(deb_preds[i] if z_deb >= z_rb else rb_preds[i])
oof_chooser = float(np.mean([jaccard(t,p) for t,p in zip(trues, chooser_preds)]))
print(f'Chooser OOF Jaccard: {oof_chooser:.5f}')

# Apply chooser to test
test_df = pd.read_csv('test.csv')
rb_test_st = np.mean([np.load(f'test_start_fold{f}.npy') for f in range(5)], axis=0)
rb_test_en = np.mean([np.load(f'test_end_fold{f}.npy') for f in range(5)], axis=0)
deb_test_st = np.mean([np.load(f'deberta_test_start_fold{f}.npy') for f in range(5)], axis=0)
deb_test_en = np.mean([np.load(f'deberta_test_end_fold{f}.npy') for f in range(5)], axis=0)

test_preds = []
for i in range(len(test_df)):
    text = test_df.iloc[i]['text']; sent = test_df.iloc[i]['sentiment']
    s_key = str(sent).strip().lower()
    if s_key=='neutral':
        test_preds.append(text); continue
    # RoBERTa decode
    sids, offs = seq_ids_offs(rb_tok, sent, text)
    p_rb, sc_rb = decode_score(rb_test_st[i], rb_test_en[i], offs, sids, text, sent, rb_lp)
    # DeBERTa decode
    sids2, offs2 = seq_ids_offs(deb_tok, sent, text)
    lp = deb_lp_by_sent.get(s_key, 0.0)
    p_deb, sc_deb = decode_score(deb_test_st[i], deb_test_en[i], offs2, sids2, text, sent, lp)
    # standardize using OOF stats
    z_rb = zscore(sc_rb, stats_rb.get(s_key, None))
    z_deb = zscore(sc_deb, stats_deb.get(s_key, None))
    test_preds.append(p_deb if z_deb >= z_rb else p_rb)

sub_ens = pd.DataFrame({'textID': test_df['textID'], 'selected_text': test_preds})
sub_ens.to_csv('submission_ensemble.csv', index=False)
print('Wrote submission_ensemble.csv. Head:\n', sub_ens.head().to_string(index=False))

# Optionally set as final submission
# pd.read_csv('submission_ensemble.csv').to_csv('submission.csv', index=False)
# print('Updated submission.csv with chooser ensemble')

Building chooser ensemble (OOF-calibrated z-scores)...


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/.pip-target/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Chooser OOF Jaccard: 0.71106


Wrote submission_ensemble.csv. Head:
     textID                                                                                       selected_text
80a1e6bc32                                                                                      I made my wish
863097735d                                                                                              sucks!
264cd5277f             tired and didn`t really have an exciting Saturday.  oh well, hope it`s better tomorrow.
baee1e6ffc                                                              i`ve been eating cheetos all morning..
67d06a8dee  haiiii sankQ i`m fineee ima js get a checkup cos my rib hurts LOL idk but i shall be fine ~ thanks


In [12]:
# DeBERTa-v3-large 5-fold training (priority) with OOF/test logits caching
import os, gc, math, json, time, numpy as np, pandas as pd, torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments, default_data_collator

MODEL_NAME = 'microsoft/deberta-v3-large'
print('Loading tokenizer/model:', MODEL_NAME, flush=True)
tok_large = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LEN = 128
BATCH_SIZE = 12
GRAD_ACCUM = 6  # effective batch ~72
EPOCHS = 2
LR = 1e-5
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
GRAD_CLIP = 1.0

def build_encodings_df_large(df: pd.DataFrame, include_labels: bool = True):
    enc_list = []
    for _, r in df.iterrows():
        sel = r['selected_text'] if include_labels else None
        enc, _ = tokenize_and_align(tok_large, r['text'], r['sentiment'], sel, max_len=MAX_LEN)
        enc['text'] = r['text']
        enc['sentiment'] = r['sentiment']
        enc_list.append(enc)
    keys = enc_list[0].keys()
    out = {}
    for k in keys:
        vals = [e[k] for e in enc_list]
        if k in ('text','sentiment'): out[k] = vals
        else: out[k] = np.array(vals, dtype=object if k=='offset_mapping' else None)
    return out

class QADatasetLarge(torch.utils.data.Dataset):
    def __init__(self, enc, with_labels): self.enc, self.with_labels = enc, with_labels
    def __len__(self): return len(self.enc['input_ids'])
    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.enc['input_ids'][idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.enc['attention_mask'][idx], dtype=torch.long),
        }
        if self.with_labels:
            item['start_positions'] = torch.tensor(self.enc['start_positions'][idx], dtype=torch.long)
            item['end_positions'] = torch.tensor(self.enc['end_positions'][idx], dtype=torch.long)
        return item

def get_seq_ids_offsets_large(sentiment: str, text: str):
    tmp = tok_large(str(sentiment), str(text), max_length=MAX_LEN, padding='max_length', truncation='only_second',
                    add_special_tokens=True, return_offsets_mapping=True)
    return tmp.sequence_ids(), tmp['offset_mapping']

def decode_span_large(start_logits, end_logits, offsets, sequence_ids, text, sentiment, length_penalty_per_char=0.003):
    if str(sentiment).strip().lower() == 'neutral': return text
    ctx = [i for i, sid in enumerate(sequence_ids) if sid == 1]
    if not ctx: return text
    sl = np.asarray(start_logits, dtype=np.float32); el = np.asarray(end_logits, dtype=np.float32)
    mask = np.zeros_like(sl, dtype=bool); mask[np.array(ctx)] = True
    sl[~mask] = -np.inf; el[~mask] = -np.inf
    best, bi, bj = -1e9, ctx[0], ctx[0]
    valid = [i for i in ctx if offsets[i][1] > offsets[i][0]]
    if not valid: return text
    for i in valid:
        for j in valid:
            if j < i: continue
            span_len = offsets[j][1] - offsets[i][0]
            sc = float(sl[i]) + float(el[j]) - length_penalty_per_char * float(span_len)
            if sc > best: best, bi, bj = sc, i, j
    s_char, e_char = offsets[bi][0], offsets[bj][1]
    pred = text[s_char:e_char].strip()
    return pred if pred else text

def jaccard_batch_fast(trues, preds):
    def jac(a,b):
        sa, sb = set(str(a).split()), set(str(b).split())
        if not sa and not sb: return 1.0
        if not sa or not sb: return 0.0
        inter = len(sa & sb); union = len(sa | sb)
        return inter/union if union else 0.0
    return float(np.mean([jac(t,p) for t,p in zip(trues, preds)]))

oof_rows_l, start_logits_folds_l, end_logits_folds_l = [], [], []
for fold in range(5):
    t0 = time.time(); print(f'\n===== DeBERTa-v3-large Fold {fold} =====', flush=True)
    trn_df = train_clean[train_clean.fold != fold].reset_index(drop=True)
    val_df = train_clean[train_clean.fold == fold].reset_index(drop=True)
    print('Train/Val sizes:', len(trn_df), len(val_df))

    trn_enc = build_encodings_df_large(trn_df, include_labels=True)
    val_enc = build_encodings_df_large(val_df, include_labels=True)
    train_ds = QADatasetLarge(trn_enc, with_labels=True)
    val_ds = QADatasetLarge(val_enc, with_labels=True)

    model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

    def compute_metrics(eval_pred):
        start_logits, end_logits = eval_pred.predictions
        preds, trues = [], list(val_df['selected_text'].astype(str).values)
        for i in range(len(val_df)):
            text, sentiment = val_df.iloc[i]['text'], val_df.iloc[i]['sentiment']
            seq_ids, offs = get_seq_ids_offsets_large(sentiment, text)
            preds.append(decode_span_large(start_logits[i], end_logits[i], offs, seq_ids, text, sentiment))
        return {'jaccard': jaccard_batch_fast(trues, preds)}

    args = TrainingArguments(
        output_dir=f'./outputs_{MODEL_NAME.replace("/","_")}_fold{fold}',
        evaluation_strategy='epoch', save_strategy='epoch',
        load_best_model_at_end=True, metric_for_best_model='jaccard', greater_is_better=True,
        per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2,
        gradient_accumulation_steps=GRAD_ACCUM, num_train_epochs=EPOCHS, fp16=True,
        learning_rate=LR, weight_decay=WEIGHT_DECAY, warmup_ratio=WARMUP_RATIO, lr_scheduler_type='cosine',
        max_grad_norm=GRAD_CLIP, dataloader_num_workers=2, logging_steps=50, save_total_limit=1, seed=SEED, report_to=[]
    )

    trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds,
                      tokenizer=tok_large, data_collator=default_data_collator, compute_metrics=compute_metrics)
    train_out = trainer.train()
    print('Best model metrics:', train_out.metrics, flush=True)
    save_dir = f'deberta_large_fold{fold}_best'
    trainer.save_model(save_dir)
    if getattr(trainer.state, 'best_model_checkpoint', None):
        with open(os.path.join(save_dir, 'path.txt'), 'w') as f: f.write(trainer.state.best_model_checkpoint)

    val_preds = trainer.predict(val_ds)
    vsl, vel = val_preds.predictions
    start_logits_folds_l.append(vsl); end_logits_folds_l.append(vel)

    trues = list(val_df['selected_text'].astype(str).values)
    preds = []
    for i in range(len(val_df)):
        text, sentiment = val_df.iloc[i]['text'], val_df.iloc[i]['sentiment']
        seq_ids, offs = get_seq_ids_offsets_large(sentiment, text)
        preds.append(decode_span_large(vsl[i], vel[i], offs, seq_ids, text, sentiment))
        oof_rows_l.append({
            'textID': val_df.iloc[i]['textID'], 'fold': fold, 'sentiment': sentiment, 'text': text,
            'selected_text': trues[i], 'pred': preds[-1]
        })
    fj = jaccard_batch_fast(trues, preds)
    print(f'DeBERTa-v3-large Fold {fold} OOF Jaccard: {fj:.5f}; elapsed {time.time()-t0:.1f}s', flush=True)

    del trainer, model, train_ds, val_ds, trn_enc, val_enc
    gc.collect(); torch.cuda.empty_cache()

oof_df_l = pd.DataFrame(oof_rows_l)
oof_score_l = jaccard_batch_fast(oof_df_l['selected_text'].tolist(), oof_df_l['pred'].tolist())
print(f'DeBERTa-v3-large OOF Jaccard (all folds): {oof_score_l:.5f}', flush=True)
oof_df_l.to_csv('oof_deberta_v3_large.csv', index=False)
np.save('oof_start_logits_deberta_v3_large.npy', np.concatenate(start_logits_folds_l, axis=0))
np.save('oof_end_logits_deberta_v3_large.npy', np.concatenate(end_logits_folds_l, axis=0))
print('Saved DeBERTa-v3-large OOF artifacts.')

# Test inference and per-fold logits cache
test_df = test.copy().reset_index(drop=True)
test_enc = [tok_large(str(r['sentiment']), str(r['text']), max_length=MAX_LEN, padding='max_length',
                     truncation='only_second', add_special_tokens=True, return_offsets_mapping=True, return_attention_mask=True)
            for _, r in test_df.iterrows()]
test_input_ids = torch.tensor([e['input_ids'] for e in test_enc], dtype=torch.long)
test_attention_mask = torch.tensor([e['attention_mask'] for e in test_enc], dtype=torch.long)
fold_starts, fold_ends = [], []
for fold in range(5):
    print(f'DeBERTa-v3-large test inference fold {fold}...', flush=True)
    model = AutoModelForQuestionAnswering.from_pretrained(f'deberta_large_fold{fold}_best').to(device); model.eval()
    with torch.no_grad():
        bs = BATCH_SIZE
        starts, ends = [], []
        for i in range(0, len(test_df), bs):
            out = model(input_ids=test_input_ids[i:i+bs].to(device), attention_mask=test_attention_mask[i:i+bs].to(device))
            starts.append(out.start_logits.detach().cpu().numpy()); ends.append(out.end_logits.detach().cpu().numpy())
        starts, ends = np.vstack(starts), np.vstack(ends)
    np.save(f'deberta_large_test_start_fold{fold}.npy', starts); np.save(f'deberta_large_test_end_fold{fold}.npy', ends)
    fold_starts.append(starts); fold_ends.append(ends)
    del model; gc.collect(); torch.cuda.empty_cache()

avg_st = np.mean(fold_starts, axis=0); avg_en = np.mean(fold_ends, axis=0)
test_preds = []
for i in range(len(test_df)):
    text, sentiment = test_df.iloc[i]['text'], test_df.iloc[i]['sentiment']
    seq_ids, offs = get_seq_ids_offsets_large(sentiment, text)
    test_preds.append(decode_span_large(avg_st[i], avg_en[i], offs, seq_ids, text, sentiment))
sub_l = pd.DataFrame({'textID': test_df['textID'], 'selected_text': test_preds})
sub_l.to_csv('submission_deberta_large.csv', index=False)
print('Wrote submission_deberta_large.csv Head:\n', sub_l.head().to_string(index=False))
print('DeBERTa-v3-large run complete.')

Loading tokenizer/model: microsoft/deberta-v3-large



===== DeBERTa-v3-large Fold 0 =====


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/.pip-target/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Train/Val sizes: 19784 4947


Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

DeBERTa-v3-large test inference fold 1...


DeBERTa-v3-large test inference fold 2...


DeBERTa-v3-large test inference fold 3...


DeBERTa-v3-large test inference fold 4...


Wrote submission_deberta_large.csv Head:
     textID                                                                                       selected_text
80a1e6bc32                                                                                                wish
863097735d                                                                                   gosh today sucks!
264cd5277f             tired and didn`t really have an exciting Saturday.  oh well, hope it`s better tomorrow.
baee1e6ffc                                                              i`ve been eating cheetos all morning..
67d06a8dee  haiiii sankQ i`m fineee ima js get a checkup cos my rib hurts LOL idk but i shall be fine ~ thanks
DeBERTa-v3-large run complete.


In [13]:
# Boundary-cleaning post-processing for DeBERTa (staged) — evaluate on OOF and prep test decode
import numpy as np, pandas as pd, json, time, string
from transformers import AutoTokenizer

print('Staging boundary-cleaning PP for DeBERTa-v3-base (OOF eval + test writer).', flush=True)

MAX_LEN = 128
tok_deb = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base', use_fast=True)

def get_seq_ids_offs_deb(sentiment: str, text: str):
    tmp = tok_deb(str(sentiment), str(text), max_length=MAX_LEN, padding='max_length', truncation='only_second',
                  add_special_tokens=True, return_offsets_mapping=True)
    return tmp.sequence_ids(), tmp['offset_mapping']

def decode_best_indices(sl_row, el_row, offs, seq_ids, lp_char: float):
    ctx = [i for i, sid in enumerate(seq_ids) if sid == 1]
    if not ctx: return None, None
    sl = np.asarray(sl_row, dtype=np.float32).copy()
    el = np.asarray(el_row, dtype=np.float32).copy()
    mask = np.zeros_like(sl, dtype=bool); mask[np.array(ctx)] = True
    sl[~mask] = -np.inf; el[~mask] = -np.inf
    valid = [i for i in ctx if offs[i][1] > offs[i][0]]
    if not valid: return None, None
    best, bi, bj = -1e9, valid[0], valid[0]
    for i in valid:
        oi0 = offs[i][0]
        for j in valid:
            if j < i: continue
            sc = float(sl[i]) + float(el[j]) - lp_char * float(offs[j][1] - oi0)
            if sc > best: best, bi, bj = sc, i, j
    return bi, bj

BOUNDARY_PUNCT_LEFT = set('"\'`([{' )
BOUNDARY_PUNCT_RIGHT = set('")]}.,!?;:…')

def expand_to_word_boundaries(text: str, s_char: int, e_char: int):
    # Expand if cut through alnum on edges
    if s_char is None or e_char is None: return 0, len(text)
    n = len(text); i, j = max(0, s_char), min(n, e_char)
    if i < j:
        # expand left if mid-word
        if i > 0 and i < n and text[i].isalnum() and text[i-1].isalnum():
            while i > 0 and text[i-1].isalnum():
                i -= 1
        # expand right if mid-word
        if j > 0 and j < n and text[j-1].isalnum() and (text[j].isalnum() if j < n else False):
            while j < n and text[j].isalnum():
                j += 1
    return i, j

def trim_outer_ws_punct(text: str, s_char: int, e_char: int):
    i, j = s_char, e_char
    # strip spaces first
    while i < j and text[i].isspace(): i += 1
    while j > i and text[j-1].isspace(): j -= 1
    # then gentle punctuation trim (avoid eating core word chars)
    while i < j and text[i] in BOUNDARY_PUNCT_LEFT: i += 1
    while j > i and text[j-1] in BOUNDARY_PUNCT_RIGHT: j -= 1
    return i, j

def decode_with_boundary_clean(sl_row, el_row, offs, seq_ids, text: str, sentiment: str, lp_char: float):
    s_key = str(sentiment).strip().lower()
    if s_key == 'neutral':
        return text
    bi, bj = decode_best_indices(sl_row, el_row, offs, seq_ids, lp_char)
    if bi is None or bj is None:
        return text
    s_char, e_char = offs[bi][0], offs[bj][1]
    s_char, e_char = expand_to_word_boundaries(text, s_char, e_char)
    s_char, e_char = trim_outer_ws_punct(text, s_char, e_char)
    pred = text[s_char:e_char]
    pred = pred if pred.strip() else text
    return pred

def jaccard_mean(y_true, y_pred):
    def jac(a,b):
        sa, sb = set(str(a).split()), set(str(b).split())
        if not sa and not sb: return 1.0
        if not sa or not sb: return 0.0
        inter = len(sa & sb); union = len(sa | sb)
        return inter/union if union else 0.0
    return float(np.mean([jac(t,p) for t,p in zip(y_true, y_pred)]))

def eval_oof_boundary_gain():
    # Load OOF and logits
    oof_df = pd.read_csv('oof_deberta_v3_base.csv')
    sl = np.load('oof_start_logits_deberta_v3_base.npy')
    el = np.load('oof_end_logits_deberta_v3_base.npy')
    # Load tuned lp params if available
    try:
        ppj = json.load(open('pp_params_deberta.json'))
        lp_by_sent = ppj.get('lp', {'positive': 0.008, 'negative': 0.002})
    except Exception:
        lp_by_sent = {'positive': 0.008, 'negative': 0.002}
    preds_no_bc, preds_bc = [], []
    t0 = time.time()
    for i in range(len(oof_df)):
        r = oof_df.iloc[i]
        s_key = str(r['sentiment']).strip().lower()
        seq_ids, offs = get_seq_ids_offs_deb(r['sentiment'], r['text'])
        lp = lp_by_sent.get(s_key, 0.0)
        # baseline decode (trim only .strip() implicitly via slicing below)
        bi, bj = decode_best_indices(sl[i], el[i], offs, seq_ids, lp)
        if bi is None or bj is None or s_key=='neutral':
            preds_no_bc.append(r['text'])
        else:
            s_char, e_char = offs[bi][0], offs[bj][1]
            pred0 = r['text'][s_char:e_char].strip()
            preds_no_bc.append(pred0 if pred0 else r['text'])
        # boundary-cleaned
        preds_bc.append(decode_with_boundary_clean(sl[i], el[i], offs, seq_ids, r['text'], r['sentiment'], lp))
    score0 = jaccard_mean(oof_df['selected_text'].tolist(), preds_no_bc)
    score1 = jaccard_mean(oof_df['selected_text'].tolist(), preds_bc)
    print(f'DeBERTa OOF no-boundary: {score0:.5f} | with-boundary: {score1:.5f} | delta: {score1-score0:+.5f} | n={len(oof_df)} | {time.time()-t0:.1f}s', flush=True)
    return lp_by_sent, score0, score1

def write_test_with_boundary_clean():
    test_df = pd.read_csv('test.csv')
    # Load per-fold test logits and average (already cached by cell 6)
    st = np.mean([np.load(f'deberta_test_start_fold{f}.npy') for f in range(5)], axis=0)
    en = np.mean([np.load(f'deberta_test_end_fold{f}.npy') for f in range(5)], axis=0)
    try:
        lp_by_sent, _, _ = eval_oof_boundary_gain()
    except Exception:
        lp_by_sent = {'positive': 0.008, 'negative': 0.002}
    preds = []
    for i in range(len(test_df)):
        text, sent = test_df.iloc[i]['text'], test_df.iloc[i]['sentiment']
        s_key = str(sent).strip().lower()
        if s_key == 'neutral':
            preds.append(text); continue
        seq_ids, offs = get_seq_ids_offs_deb(sent, text)
        preds.append(decode_with_boundary_clean(st[i], en[i], offs, seq_ids, text, sent, lp_by_sent.get(s_key, 0.0)))
    sub = pd.DataFrame({'textID': test_df['textID'], 'selected_text': preds})
    sub.to_csv('submission_deberta_boundary.csv', index=False)
    print('Wrote submission_deberta_boundary.csv. Head:\n', sub.head().to_string(index=False), flush=True)

print('Boundary-cleaning utilities ready. After current training finishes, run:',
      '\n- eval_oof_boundary_gain() to verify gain',
      '\n- write_test_with_boundary_clean() to produce submission_deberta_boundary.csv', flush=True)

Staging boundary-cleaning PP for DeBERTa-v3-base (OOF eval + test writer).


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/.pip-target/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Boundary-cleaning utilities ready. After current training finishes, run: 
- eval_oof_boundary_gain() to verify gain 
- write_test_with_boundary_clean() to produce submission_deberta_boundary.csv


In [14]:
# DeBERTa-v3 base+large logit blending (OOF tune weights; optional boundary cleaning); apply to test
import numpy as np, pandas as pd, json, time
from transformers import AutoTokenizer

print('Staging DeBERTa-v3 base+large blending...', flush=True)

MAX_LEN = 128
tok_deb = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base', use_fast=True)

def get_seq_ids_offs(sentiment: str, text: str):
    tmp = tok_deb(str(sentiment), str(text), max_length=MAX_LEN, padding='max_length', truncation='only_second',
                  add_special_tokens=True, return_offsets_mapping=True)
    return tmp.sequence_ids(), tmp['offset_mapping']

# Reuse boundary-clean helpers from Cell 13 if defined; else define minimal no-op wrappers
try:
    expand_to_word_boundaries
except NameError:
    def expand_to_word_boundaries(text, s_char, e_char):
        n = len(text); i, j = max(0, s_char or 0), min(n, e_char or n)
        return i, j
try:
    trim_outer_ws_punct
except NameError:
    def trim_outer_ws_punct(text, s_char, e_char):
        i, j = s_char, e_char
        while i < j and text[i].isspace(): i += 1
        while j > i and text[j-1].isspace(): j -= 1
        return i, j

def decode_span_from_logits(sl_row, el_row, offs, seq_ids, text: str, sentiment: str, lp_char: float, boundary_clean: bool=False):
    s_key = str(sentiment).strip().lower()
    if s_key == 'neutral':
        return text
    ctx = [i for i, sid in enumerate(seq_ids) if sid == 1]
    if not ctx: return text
    sl = np.asarray(sl_row, dtype=np.float32).copy()
    el = np.asarray(el_row, dtype=np.float32).copy()
    mask = np.zeros_like(sl, dtype=bool); mask[np.array(ctx)] = True
    sl[~mask] = -np.inf; el[~mask] = -np.inf
    valid = [i for i in ctx if offs[i][1] > offs[i][0]]
    if not valid: return text
    best, bi, bj = -1e9, valid[0], valid[0]
    for i in valid:
        oi0 = offs[i][0]
        for j in valid:
            if j < i: continue
            sc = float(sl[i]) + float(el[j]) - lp_char * float(offs[j][1] - oi0)
            if sc > best: best, bi, bj = sc, i, j
    s_char, e_char = offs[bi][0], offs[bj][1]
    if boundary_clean:
        s_char, e_char = expand_to_word_boundaries(text, s_char, e_char)
        s_char, e_char = trim_outer_ws_punct(text, s_char, e_char)
    pred = text[s_char:e_char]
    pred = pred if pred.strip() else text
    return pred

def jaccard_mean(y_true, y_pred):
    def jac(a,b):
        sa, sb = set(str(a).split()), set(str(b).split())
        if not sa and not sb: return 1.0
        if not sa or not sb: return 0.0
        inter = len(sa & sb); union = len(sa | sb)
        return inter/union if union else 0.0
    return float(np.mean([jac(t,p) for t,p in zip(y_true, y_pred)]))

def blend_oof_and_eval(weights=(0.5,0.6,0.7,0.8,0.9), boundary_options=(False, True)):
    # Load OOF artifacts
    oof_df = pd.read_csv('oof_deberta_v3_base.csv')
    sl_b = np.load('oof_start_logits_deberta_v3_base.npy'); el_b = np.load('oof_end_logits_deberta_v3_base.npy')
    sl_l = np.load('oof_start_logits_deberta_v3_large.npy'); el_l = np.load('oof_end_logits_deberta_v3_large.npy')
    assert sl_b.shape == sl_l.shape == el_b.shape == el_l.shape, 'OOF logits shape mismatch between base and large'
    # Load tuned lp per sentiment
    try:
        params = json.load(open('pp_params_deberta.json'))
        lp_by_sent = params.get('lp', {'positive': 0.008, 'negative': 0.002})
    except Exception:
        lp_by_sent = {'positive': 0.008, 'negative': 0.002}
    y_true = oof_df['selected_text'].astype(str).tolist()
    best = (-1.0, None, None)
    for w in weights:
        sl = w*sl_l + (1.0-w)*sl_b
        el = w*el_l + (1.0-w)*el_b
        for bc in boundary_options:
            preds = []
            t0 = time.time()
            for i in range(len(oof_df)):
                r = oof_df.iloc[i]
                s_key = str(r['sentiment']).strip().lower()
                seq_ids, offs = get_seq_ids_offs(r['sentiment'], r['text'])
                lp = lp_by_sent.get(s_key, 0.0)
                preds.append(decode_span_from_logits(sl[i], el[i], offs, seq_ids, r['text'], r['sentiment'], lp, boundary_clean=bc))
            sc = jaccard_mean(y_true, preds)
            print(f'Blend w={w:.2f} boundary={bc} -> OOF {sc:.5f} in {time.time()-t0:.1f}s', flush=True)
            if sc > best[0]: best = (sc, w, bc)
    print('Best blend:', best, flush=True)
    return best

def apply_blend_to_test(weight: float, boundary_clean: bool):
    test_df = pd.read_csv('test.csv')
    st_b = np.mean([np.load(f'deberta_test_start_fold{f}.npy') for f in range(5)], axis=0)
    en_b = np.mean([np.load(f'deberta_test_end_fold{f}.npy') for f in range(5)], axis=0)
    st_l = np.mean([np.load(f'deberta_large_test_start_fold{f}.npy') for f in range(5)], axis=0)
    en_l = np.mean([np.load(f'deberta_large_test_end_fold{f}.npy') for f in range(5)], axis=0)
    assert st_b.shape == st_l.shape == en_b.shape == en_l.shape == (len(test_df), st_b.shape[1])
    try:
        params = json.load(open('pp_params_deberta.json'))
        lp_by_sent = params.get('lp', {'positive': 0.008, 'negative': 0.002})
    except Exception:
        lp_by_sent = {'positive': 0.008, 'negative': 0.002}
    st = weight*st_l + (1.0-weight)*st_b
    en = weight*en_l + (1.0-weight)*en_b
    preds = []
    for i in range(len(test_df)):
        text, sent = test_df.iloc[i]['text'], test_df.iloc[i]['sentiment']
        s_key = str(sent).strip().lower()
        if s_key == 'neutral':
            preds.append(text); continue
        seq_ids, offs = get_seq_ids_offs(sent, text)
        preds.append(decode_span_from_logits(st[i], en[i], offs, seq_ids, text, sent, lp_by_sent.get(s_key, 0.0), boundary_clean=boundary_clean))
    sub = pd.DataFrame({'textID': test_df['textID'], 'selected_text': preds})
    out_path = f'submission_deberta_blend_w{weight:.2f}_bc{int(boundary_clean)}.csv'
    sub.to_csv(out_path, index=False)
    print('Wrote', out_path, 'Head:\n', sub.head().to_string(index=False), flush=True)
    return out_path

print('Blend utilities ready. After large training finishes and saves logits, run:',
      '\n- best = blend_oof_and_eval()  # pick (score, w, boundary)',
      '\n- apply_blend_to_test(best[1], best[2])  # write submission', flush=True)

Staging DeBERTa-v3 base+large blending...


Blend utilities ready. After large training finishes and saves logits, run: 
- best = blend_oof_and_eval()  # pick (score, w, boundary) 
- apply_blend_to_test(best[1], best[2])  # write submission


In [ ]:
# Run DeBERTa base+large blend grid on OOF; apply best to test and (optionally) set submission.csv
import pandas as pd, shutil
best_score, best_w, best_bc = blend_oof_and_eval(weights=(0.5,0.6,0.7,0.8,0.9), boundary_options=(False, True))
print(f'Best blended OOF: {best_score:.5f} with w={best_w} boundary_clean={best_bc}', flush=True)
out_path = apply_blend_to_test(best_w, best_bc)
print('Blend submission path:', out_path, flush=True)
# If blend beats prior best single-model OOF (0.71239), set as final submission
if best_score > 0.71239:
    pd.read_csv(out_path).to_csv('submission.csv', index=False)
    print('submission.csv updated with blended model (OOF improved).', flush=True)
else:
    print('Blended OOF did not beat 0.71239; keeping current submission.csv.', flush=True)

Blend w=0.50 boundary=False -> OOF 0.71502 in 4.9s
